<a href="https://colab.research.google.com/github/Alecia113/NLP-Emotional-analysis/blob/main/1_1_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id = '1lTD6bgRkmwguGAr30v-r0KBPdtnVneLb'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('testing_data.pkl')  

id = '1pCUdlZMoj99UZHtqFeza86fvVQfFmDFX'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('training_data.pkl')  

import pickle #必要的
training_data = pickle.load(open("training_data.pkl","rb")) #必要的
testing_data = pickle.load(open("testing_data.pkl","rb")) #必要的

print("------------------------------------")
print("Size of training dataset: {0}".format(len(training_data))) #8000
print("Size of testing dataset: {0}".format(len(testing_data))) #2000
print("------------------------------------")

print("------------------------------------")
print("Sample Data")
print("LABEL: {0} / SENTENCE: {1}".format(training_data[0][0], training_data[0][1]))
print("------------------------------------")
#LABEL: neg / SENTENCE: hopeless for tmr :(

------------------------------------
Size of training dataset: 8000
Size of testing dataset: 2000
------------------------------------
------------------------------------
Sample Data
LABEL: neg / SENTENCE: hopeless for tmr :(
------------------------------------


In [2]:
import torch
from random import shuffle
import pprint
from nltk.tokenize import TweetTokenizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords as sw
import re

from nltk.tokenize import word_tokenize
from collections import Counter
import math
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


train_data = [training_data[i][1] for i in range(len(training_data))]
train_label = [training_data[i][0] for i in range(len(training_data))]
test_data = [testing_data[i][1] for i in range(len(testing_data))]  
test_label = [testing_data[i][0] for i in range(len(testing_data))] #neg pos


zipped = zip(train_data,train_label)  
Zipp = list(zipped) 
shuffle(Zipp)
train_data = [context[0] for context in Zipp]
train_label = [context[1] for context in Zipp]


def preprocessing(data_text):
  text = [s.lower() for s in data_text]


  def remove_punctuation_re(x):
      # Please complete this
      x1 = re.sub(r'[^\w\s]','',x)
      x2 = re.sub(r'\d','',x1)
      return x2

  text_re = [remove_punctuation_re(s) for s in text]

  tknzr = TweetTokenizer()
  text_t=[]                                    #完全每句话的分词的集合。train_t[0][0]每个单词
  for s in text_re:
    text_re = tknzr.tokenize(s)              #每句话分别作了分词 text_train[0]第一句话的分词
    text_t.append(text_re)  
  '''表情的保留方式  因为字典中没表情，所以就不用表情了
  def remove(x):
    t = []
    for i in range(len(x)):
      t_sub = []        #是直接用空列表代替了
      for j in range(len(x[i])):
        if len(x[i][j])==0:
          continue
        if x[i][j] == " ":
          continue 
        else:
          x[i][j] = re.sub(r'\d','',x[i][j])                  #删除0-9
          if len(x[i][j]) == 1 or len(x[i][j]) > 3:           #and + or 问题。
            x[i][j] = re.sub(r'[^\w\s+]','', x[i][j])
          if len(x[i][j]) == 0:
            continue 
          else:
            t_sub.append(x[i][j]) 
      t.append(t_sub) 
    return t

  new_text = remove(text_t)
  '''



  stop_words = sw.words()
  sww = sw.words()
  text_stop=[]    #8000
  for tokens in text_t:
      filtered_sentence = [w for w in tokens if not w in stop_words]
      text_stop.append(filtered_sentence)
  return text_stop

pre_train= preprocessing(train_data)  #对应后面，后面还得改一下 train_stem ;test_stem
pre_test =preprocessing(test_data)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
'''
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

def lemmatize(past_text):
  lemmatizer = WordNetLemmatizer()

  text_le = []
  for tokens in past_text:
    lemma_sentence = [lemmatizer.lemmatize(w) for w in tokens ]
    text_le.append(lemma_sentence)
  return text_le

lem_train = lemmatize(pre_train)  #nobodies-->nobody
lem_test = lemmatize(pre_test)
'''

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:

# stem
from nltk.stem.porter import *
def stemming(past_text):
  stemmer = PorterStemmer()
  train_stem = []
  for i in range(len(past_text)):
    singles = []
    for plural in past_text[i]:
      singles.append(stemmer.stem(plural))
    train_stem.append(singles)
  return train_stem

stem_train = stemming(pre_train)  #nobodies -->nobodi
stem_test = stemming(pre_test)


In [ ]:
#为什么要stem
II = torch.load('save_stem.PATH')  # 'save_lem.PATH'
O=model.load_state_dict(II['model_state_dict']) #成功
optimizer.load_state_dict(II['optimizer_state_dict'])
PP = II['epoch'] #19
LL = II['loss'] #0.6927
P2 = II['predicted2']   #tensor([0, 1, 0,  ..., 0, 1, 0])
O2 = II['outputs2']   
'''
tensor([[ 0.1570, -0.1038],
        [-0.2550,  0.3335],
        [ 0.1559, -0.0992],
        ...,
        [ 0.1649, -0.1156],
        [-0.1996,  0.2628],
        [ 0.1186, -0.0533]], requires_grad=True)
        '''
P = II['predicted'] #tensor([0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,// 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
ACC = II['acc'] #0.61  看起来是存了期中的一个
TL = II['train_loss'] #6.557632803916931  看起来是存了期中的一个
#不再存直接跑是上次的状态
print(PP)
print(LL)
print(P2)
print(O2)
print(P)
print(ACC)
print(TL)

想测试哪个预处理好用。

In [4]:
sentences = stem_train + stem_test      #

from gensim.models import Word2Vec
wv_cbow_model = Word2Vec(sentences= sentences, size=100, window=5, min_count=2, workers=2, sg=0)
wv_cbow_model.save("cbow.model")

cbow = Word2Vec.load("./cbow.model")  


In [11]:
from google.colab import drive
drive.mount('/content/drive') 

not_words = [w.strip() for w in open('/content/drive/MyDrive/NLP/A1/E/opinion-lexicon-English/negative-words.txt', 'r', encoding="ISO-8859-1").readlines()]
#处理文件 -
j = -1
for i in not_words:
  j +=1
  if i == '2-faced':
    break
neg = not_words[j:]

#处理文件+
sure_words = [w.strip() for w in open('/content/drive/MyDrive/NLP/A1/E/opinion-lexicon-English/positive-words.txt', 'r', encoding="ISO-8859-1").readlines()]

j = -1
for i in sure_words:
  j +=1
  if i == 'a+':
    break

pos = sure_words[j:]


nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
'''
def lem(past_text):
  lemmatizer = WordNetLemmatizer()

  text_le = []

  lemma_sentence = [lemmatizer.lemmatize(w) for w in past_text ]
  text_le.append(lemma_sentence)
  return text_le
neg_new = lem(neg)
pos_new = lem(pos)
'''


from nltk.stem.porter import *
def stem(past_text):
  stemmer = PorterStemmer()
  text_le = []
  stemmer_sentence = [stemmer.stem(plural) for plural in past_text ]
  text_le.append(stemmer_sentence)
  return text_le

neg_new = stem(neg)
pos_new = stem(pos)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
#只是原来的，因为neg就是单词了。不能再neg[0]
def match(text):
  sum_match= [] #train_match
  for i in range(len(text)):  #train_stem  取出一句话
    match = []
    for j in range(len(text[i])):   #取出一句话中的单词
      if text[i][j] in neg_new[0]:    #neg_stem  第几句话的第几个单词
        match.append(1)   #neg 1
      elif text[i][j] in pos_new[0]:  #pos_stem
        match.append(2) #pos 2
      else:
        match.append(0)
    sum_match.append(match)
  return sum_match
train_match = match(stem_train)  #这在不断的改变 train_stem  原本的话都是000
test_match = match(stem_test)
#只有stem好用
#print(train_match)
#print(test_match[:5])
#就是把我原本的要训练的话，纷纷用012 表示出来。每个词告诉他是积极消极还是不在

[[0, 0, 0, 0], [0, 2, 0, 0, 0], [0, 0], [0, 2, 0, 2, 0, 0, 0, 1, 0, 0, 2], [0, 0, 0, 2, 0, 2, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 2, 0, 0, 0, 0, 0, 0, 1, 2, 2], [2, 0, 0, 2, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0], [0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 2, 0, 0], [0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 2], [0, 0], [0, 0, 0, 0], [0, 0], [0, 0, 2, 0, 2, 0], [0, 0, 0, 0, 2], [0, 0, 0, 0], [1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2], [0, 0, 0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0], [0, 2], [0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0], [1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 2, 0], [0, 2, 0, 0, 0, 0, 0], [0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0], [0, 2, 0, 2, 0, 0, 2], [0, 2, 0], [0, 0], [0, 0, 0, 2, 0, 1, 0, 0], [0, 0, 0, 2, 2, 2, 0], [2, 0, 2, 0, 2, 0, 0], [0, 1, 0], [1, 2, 0, 2, 0], [2, 0, 0, 2, 0], 

In [ ]:
lem_test

In [16]:
#实现的是concat的功能。
'''
大改动
'''
def input(text,text_match):
  DF = {}

  for tokensized_doc in text: 
      for term in np.unique(tokensized_doc):
          try:
              DF[term] +=1
          except:
              DF[term] =1
  input = []
  for w in range(len(text)):                #8000 0-7999
    num = 0
    new = []
    for t in text[w]:                       #一句话 #t就是这个单词  #13
      match = []                            #目前这个就是第一句话的match
      if DF[t] >= 2:                         #0-12     这个min_count
        word_vec = wv_cbow_model[t].tolist()
        match.append(text_match[w][num])     #w 012 []   #IndexError: list index out of range 后面的num问题
        new_embedding = word_vec + match       #sent_embedding
        new.append(new_embedding)
      num += 1
      if num >= len(text_match[w]):
        break
    input.append(new)                          # input == train_embedding
                                                #print(input[0][0]) #input 8000一句话 3一个词 101 vec+0、1

  return input

input_train = input(stem_train,train_match)         #train_stem变化
input_test = input(stem_test,test_match)           #test_stem变化
#pprint.pprint(input_train[:1])
#print('\n')
#pprint.pprint(input_test[:1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [17]:
#主要是为了得到n_class
unique_labels = np.unique(train_label)

lEnc = LabelEncoder()
label_train_encoded = lEnc.fit_transform(train_label)         #lEnc.transform(unique_labels)[0]+1
label_test_encoded = lEnc.fit_transform(test_label)
n_class = len(unique_labels)                                   #主要是为了得到n_class  #n_class = np.unique(unique_labels).shape[0] #2相当于

#要确定最大长度是多少 [lab4]
doc_length_list = []                                              #会得到8000句话
sum_text = pre_train + pre_test     #train_stem + test_stem要改的
maxlength = 0
for doc in sum_text:
    doc_length_list.append(len(doc))                                  #每句话多少个分词

for index in range(len(doc_length_list)):
  if doc_length_list[index] > maxlength:
    maxlength = doc_length_list[index]
    max_index = index
#print(doc_length_list[max_index]) #21


In [18]:
#超参

seq_length = doc_length_list[max_index]           #（22）应该是我规定的最大序列长度512那种  maxlength 增加减少那种 #一句话最长多少个词
n_input = len(input_train[0][0])              #101 无所谓的反正都是101维度
n_class = len(unique_labels)                  #2

n_hidden = 200                                  #128
batch_size = 800                                #500 
total_epoch = 20                                #10
learning_rate = 0.001                           # 1e-4 == 0.001 le-3 == 0.01 #0.1  0.05


#把输入的规格都定成一样的， 变成句子长度是22，单词维度是101

def sent_encoded(input_text):
  sent_encoded = []
  for m in range(len(input_text)):                       # m 1-8000 #训练集  input_train  input_test
    
    encoded = [] 
    zero = []
    if len(input_text[m]) < seq_length:
      zero = [len(input_text[0][0])*[0]] * (seq_length - len(input_text[m]))#补充到22 最长 # 不单单加0； 还是需要变成101维度
      encoded = input_text[m] + zero
    else:
      encoded = input_text[m]
    sent_encoded.append(encoded)                                              #要重新添加。 句子已经全变成22了 

  sent_encoded = np.array(sent_encoded)
  return sent_encoded

sent_encoded_train = sent_encoded(input_train)
sent_encoded_test = sent_encoded(input_test)
#print(sent_encoded_train[:1]) #(8000, 22, 101)
#print(sent_encoded_test[:1])#(2000, 22, 101)


#还是Bi-LSTM准确度更高，因为LSTM处理了记忆丢失的问题。
'''
要写个LSTM比RNN好在哪里。
'''
# 模型Bi-RNN


'''
class Bi_RNN_Model(nn.Module):
    def __init__(self):
        super(Bi_RNN_Model, self).__init__()
        # set the bidirectional to True
        self.rnn = nn.RNN(n_input, n_hidden, batch_first =True, bidirectional=True) #, dropout = 0.5
        self.linear = nn.Linear(2*n_hidden,n_class) #n_class = 3 
        #https://stackoverflow.com/questions/60259836/cnn-indexerror-target-2-is-out-of-bounds 虽然是01分类但是，pytorch 要进行012 
    def forward(self, x):        
        x, h_n = self.rnn(x)
        # concat the last hidden state from two direction
        hidden_out = torch.cat((h_n[0,:,:],h_n[1,:,:]),1)
        output = self.linear(hidden_out)
        return output


'''

#LSTM-BI
class Bi_LSTM_Model(nn.Module):
    def __init__(self):
        super(Bi_LSTM_Model, self).__init__()
        #self.emb = nn.Embedding(vocab_size, emb_dim)
        # Initialize the Embedding layer with the lookup table we created 
        #self.emb.weight.data.copy_(torch.from_numpy(emb_table))
        # Optional: set requires_grad = False to make this lookup table untrainable
        #self.emb.weight.requires_grad = False

        self.lstm = nn.LSTM(n_input, n_hidden, batch_first =True, bidirectional=True)
        self.linear = nn.Linear(n_hidden*2, n_class)

    def forward(self, x):
        # Get the embeded tensor
        #x = self.emb(x)        
        # we will use the returned h_n of shape (num_layers * num_directions, batch, hidden_size): tensor containing the hidden state for t = seq_len.
        # details of the outputs from nn.LSTM can be found from: https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html
        lstm_out, (h_n,c_n) = self.lstm(x)
        # concat the last hidden state from two direction
        hidden_out =torch.cat((h_n[0,:,:],h_n[1,:,:]),1)
        z = self.linear(hidden_out)
        return z


'''
criterion = nn.CrossEntropyLoss()
model = Bi_RNN_Model().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
'''
#Bi-LSTM
# Move the model to GPU
criterion = nn.CrossEntropyLoss()
model = Bi_LSTM_Model().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)




In [19]:
# Please comment your code
#[lab4][lab5]
#检查输出节点数与标签数是否一致。
#并且将输出节点数num_outputs更改为标签数。
# 因为我之前设置的class 类别是 0，1；所以这里也需要改成0，1 要不然后面会出现 target超出范围的错误。
def label(label):   
  label = np.array(label) #train_label
  #neg 1 pos2
  lab = []  #label_train
  for tag in label:
    if tag == 'neg':
      lab.append(0)
    else:
      lab.append(1)
  #print(label_train)
  #print(len(label_train))
  lab = np.array(lab)
  #test_label
  return lab

label_train = label(train_label)
label_test = label(test_label)

#要开始训练。 optimizer +loss+ backward

#处理
#sent_encoded_train = sent_encoded(input_train)
#sent_encoded_test = sent_encoded(input_test)

for epoch in range(total_epoch):
    train_loss = 0
    for ind in range(0,sent_encoded_train.shape[0],batch_size):
        input_batch = sent_encoded_train[ind:min(ind+batch_size, sent_encoded_train.shape[0])] #256,22,101
        target_batch = label_train[ind:min(ind+batch_size, sent_encoded_train.shape[0])]  #256
        input_batch_torch = torch.from_numpy(input_batch).float().to(device)  #256,22,101
        target_batch_torch = torch.from_numpy(target_batch).view(-1).to(device) #256

        model.train()
        optimizer.zero_grad()
        outputs = model(input_batch_torch)   #13,512,25 #256，22，101    #256 2 #500,22,101
        loss = criterion(outputs, target_batch_torch) #256 2    256 out,y
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    predicted = torch.argmax(outputs, 1)  # 输出值为元祖获取第二个tensor        
    acc = accuracy_score(predicted.cpu().numpy(),target_batch_torch.cpu().numpy())
    print('Epoch: %d, train loss: %.5f, train_acc:%.2f'%(epoch + 1, train_loss, acc))

print('Finished Training')
#这块是在后面评估，可删
## Prediction

model.eval()
outputs2 = model(torch.from_numpy(sent_encoded_test).float().to(device)) 
predicted2 = torch.argmax(outputs2, 1)


print(classification_report(label_test, predicted2.cpu().numpy(),digits=4))
#是改变了batch
#3.59s

Epoch: 1, train loss: 6.91395, train_acc:0.58
Epoch: 2, train loss: 6.86637, train_acc:0.56
Epoch: 3, train loss: 6.83601, train_acc:0.59
Epoch: 4, train loss: 6.80309, train_acc:0.59
Epoch: 5, train loss: 6.78627, train_acc:0.58
Epoch: 6, train loss: 6.76691, train_acc:0.59
Epoch: 7, train loss: 6.75985, train_acc:0.58
Epoch: 8, train loss: 6.73669, train_acc:0.58
Epoch: 9, train loss: 6.73543, train_acc:0.57
Epoch: 10, train loss: 6.71102, train_acc:0.59
Epoch: 11, train loss: 6.68495, train_acc:0.59
Epoch: 12, train loss: 6.65546, train_acc:0.59
Epoch: 13, train loss: 6.61689, train_acc:0.60
Epoch: 14, train loss: 6.59291, train_acc:0.61
Epoch: 15, train loss: 6.58017, train_acc:0.61
Epoch: 16, train loss: 6.54387, train_acc:0.60
Epoch: 17, train loss: 6.50573, train_acc:0.62
Epoch: 18, train loss: 6.43361, train_acc:0.60
Epoch: 19, train loss: 6.41024, train_acc:0.60
Epoch: 20, train loss: 6.41260, train_acc:0.62
Finished Training
              precision    recall  f1-score   suppo

In [20]:
torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            'predicted2':predicted2,
            'outputs2':outputs2,
            'predicted':predicted,
            'acc':acc,
            'train_loss':train_loss,
            
            }, 'save_stem.PATH')



In [23]:

II = torch.load('save_stem.PATH')
O=model.load_state_dict(II['model_state_dict']) #成功
optimizer.load_state_dict(II['optimizer_state_dict'])
PP = II['epoch'] #19
LL = II['loss'] #0.6927
P2 = II['predicted2']   #tensor([0, 1, 0,  ..., 0, 1, 0])
O2 = II['outputs2']   
'''
tensor([[ 0.1570, -0.1038],
        [-0.2550,  0.3335],
        [ 0.1559, -0.0992],
        ...,
        [ 0.1649, -0.1156],
        [-0.1996,  0.2628],
        [ 0.1186, -0.0533]], requires_grad=True)
        '''
P = II['predicted'] #tensor([0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,// 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
ACC = II['acc'] #0.61  看起来是存了期中的一个
TL = II['train_loss'] #6.557632803916931  看起来是存了期中的一个
#不再存直接跑是上次的状态
print(PP)
print(LL)
print(P2)
print(O2)
print(P)
print(ACC)
print(TL)

19
tensor(0.6375, requires_grad=True)
tensor([1, 0, 0,  ..., 0, 0, 0])
tensor([[-0.3244,  0.1959],
        [ 0.2147, -0.3042],
        [ 0.1175, -0.1901],
        ...,
        [ 0.0184, -0.0527],
        [ 0.1650, -0.2153],
        [ 0.2658, -0.3594]], requires_grad=True)
tensor([0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1,
        0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0,
        0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
        0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0,
        0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1,
       

In [ ]:
def match(text):
  sum_match= [] #train_match
  for i in range(len(text)):  #train_stem  取出一句话
    match = []
    for j in range(len(text[i])):   #取出一句话中的单词
      if text[i][j] in neg[0]:    #neg_stem  第几句话的第几个单词
        match.append(1)   #neg 1
      elif text[i][j] in pos[0]:  #pos_stem
        match.append(2) #pos 2
      else:
        match.append(0)
    sum_match.append(match)
  return sum_match
train_match = match(lem_train)  #这在不断的改变 train_stem  原本的话都是000
test_match = match(lem_test)
#只有stem好用
print(train_match[:5])
print(test_match[:5])
#就是把我原本的要训练的话，纷纷用012 表示出来。每个词告诉他是积极消极还是不在

NameError: ignored

上面是各种函数。还有cbow哪个到底用到哪了？

In [ ]:
pprint.pprint(pre_train)
pprint.pprint(pre_test[:5])

In [ ]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

def lemmatize(past_text):
  lemmatizer = WordNetLemmatizer()

  text_le = []
  for tokens in past_text:
    lemma_sentence = [lemmatizer.lemmatize(w) for w in tokens ]
    text_le.append(lemma_sentence)
  return text_le

pprint.pprint(text_train_le[:5])
print('\n')
'''
text_test_le = []
for tokens in text_test_ns:
  lemma_sentence2 = [lemmatizer.lemmatize(w) for w in tokens]
  text_test_le.append(lemma_sentence2)
pprint.pprint(text_test_le[:5])
'''

In [ ]:
import torch
from random import shuffle
import pprint
from nltk.tokenize import TweetTokenizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords as sw
import re

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


train_data = [training_data[i][1] for i in range(len(training_data))]
train_label = [training_data[i][0] for i in range(len(training_data))]
test_data = [testing_data[i][1] for i in range(len(testing_data))]  
test_label = [testing_data[i][0] for i in range(len(testing_data))] #neg pos


zipped = zip(train_data,train_label)  
Zipp = list(zipped) 
shuffle(Zipp)
train_data = [context[0] for context in Zipp]
train_label = [context[1] for context in Zipp]


def preprocessing(data_text):
  text = [s.lower() for s in data_text]


  def remove_punctuation_re(x):
      # Please complete this
      x1 = re.sub(r'[^\w\s]','',x)
      x2 = re.sub(r'\d','',x1)
      return x2
  text_re = remove_punctuation_re(text)

  tknzr = TweetTokenizer()
  text_t=[]                                    #完全每句话的分词的集合。train_t[0][0]每个单词
  for s in text:
    text = tknzr.tokenize(s)              #每句话分别作了分词 text_train[0]第一句话的分词
    text_t.append(text)  


  stop_words = sw.words()
  sww = sw.words()
  text_stop=[]    #8000
  for tokens in new_text:
      filtered_sentence = [w for w in tokens if not w in stop_words]
      text_stop.append(filtered_sentence)
  return text_stop

pre_train= preprocessing(train_data)  #对应后面，后面还得改一下 train_stem ;test_stem
pre_test =preprocessing(test_data)

TEXT

In [ ]:
  def remove(x):
    t = []
    for i in range(len(x)):
      t_sub = []        #是直接用空列表代替了
      for j in range(len(x[i])):
        if len(x[i][j])==0:
          continue
        if x[i][j] == " ":
          continue 
        else:
          x[i][j] = re.sub(r'\d','',x[i][j])                  #删除0-9
          if len(x[i][j]) == 1 or len(x[i][j]) > 3:           #and + or 问题。
            x[i][j] = re.sub(r'[^\w\s+]','', x[i][j])
          if len(x[i][j]) == 0:
            continue 
          else:
            t_sub.append(x[i][j]) 
      t.append(t_sub) 
    return t

  new_text = remove(text_t)



In [ ]:





#找词根
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

text_train_le = []
for tokens in text_train_ns:
  lemma_sentence1 = [lemmatizer.lemmatize(w) for w in tokens ]
  text_train_le.append(lemma_sentence1)
pprint.pprint(text_train_le[:5])
print('\n')
text_test_le = []
for tokens in text_test_ns:
  lemma_sentence2 = [lemmatizer.lemmatize(w) for w in tokens]
  text_test_le.append(lemma_sentence2)
pprint.pprint(text_test_le[:5])




#去掉复数 stemmer 词根不需要这部； 如果用就需要在neg和pos需要stemmer操作 保留是因为更加便捷。
#如果只保留原型lec…………那个，无法与现有给定字典一一匹配。

from nltk.stem.porter import *
stemmer = PorterStemmer()
#text_train_le = [text_train_le[i] for i in range(len(text_train_le))]
#singles = [stemmer.stem(plural) for plural in text_train_le[i] for i in range(len(text_train_le))]
train_stem = []
for i in range(len(text_train_le)):
  singles1 = []
  for plural in text_train_le[i]:
    singles1.append(stemmer.stem(plural))
  train_stem.append(singles1)



#print(train_stem[:5])

test_stem = []
for i in range(len(text_test_le)):
  singles2 = []
  for plural in text_test_le[i]:
    singles2.append(stemmer.stem(plural))
  test_stem.append(singles2)

#print(test_stem[:5])


'''
import re
def remove(x):
  t = []
  for i in range(len(x)):
    t_sub = []        #是直接用空列表代替了
    for j in range(len(x[i])):
      if len(x[i][j])==0:
        continue
      if x[i][j] == " ":
        continue 
      else:
        x[i][j] = re.sub(r'\d','',x[i][j])                  #删除0-9
        if len(x[i][j]) == 1 or len(x[i][j]) > 3:           #and + or 问题。
          x[i][j] = re.sub(r'[^\w\s+]','', x[i][j])
        if len(x[i][j]) == 0:
          continue 
        else:
          t_sub.append(x[i][j]) 
    t.append(t_sub) 
  return t
'''
text_train = remove(train_t)
text_test = remove(test_t)

##停用词  删除那些可有可无不影响句意比如a;an;the 但是又数目非常多的词。
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords as sw
stop_words = sw.words()
sww = sw.words()

text_train_le=[]    #8000
for tokens in text_train:
    filtered_sentence1 = [w for w in tokens if not w in stop_words]
    text_train_le.append(filtered_sentence1)


text_test_le=[]   #2000
for tokens in text_test:
  filtered_sentence2 = [w for w in tokens if not w in stop_words]
  text_test_le.append(filtered_sentence2)

#去掉复数 stemmer 词根不需要这部； 如果用就需要在neg和pos需要stemmer操作 保留是因为更加便捷。
#如果只保留原型lec…………那个，无法与现有给定字典一一匹配。

from nltk.stem.porter import *
stemmer = PorterStemmer()
#text_train_le = [text_train_le[i] for i in range(len(text_train_le))]
#singles = [stemmer.stem(plural) for plural in text_train_le[i] for i in range(len(text_train_le))]
train_stem = []
for i in range(len(text_train_le)):
  singles1 = []
  for plural in text_train_le[i]:
    singles1.append(stemmer.stem(plural))
  train_stem.append(singles1)

#print(train_stem[:5])

test_stem = []
for i in range(len(text_test_le)):
  singles2 = []
  for plural in text_test_le[i]:
    singles2.append(stemmer.stem(plural))
  test_stem.append(singles2)

#print(test_stem[:5])



In [ ]:
# Please comment your code
#否定词"ISO-8859-1"==UTF-8
'''
这里需要具体说明把文件丢到哪里
'''
#核心想法，我去看了看这个文件，因为有一些文本介绍。我找到了每个文件的第一个词，然后把这些词重新取出来放到一个list中。
from google.colab import drive
drive.mount('/content/drive') #必须要挂载
not_words = [w.strip() for w in open('/content/drive/MyDrive/NLP/A1/E/opinion-lexicon-English/negative-words.txt', 'r', encoding="ISO-8859-1").readlines()]
#处理文件
j = -1
for i in not_words:
  j +=1
  if i == '2-faced':
    break
neg = not_words[j:]

#处理文件
sure_words = [w.strip() for w in open('/content/drive/MyDrive/NLP/A1/E/opinion-lexicon-English/positive-words.txt', 'r', encoding="ISO-8859-1").readlines()]

j = -1
for i in sure_words:
  j +=1
  if i == 'a+':
    break

pos = sure_words[j:]

#neg pos 词表
#要对他们做stem因为训练集做了stem

'''
前面要是后来测试发现不需要这里那就不用了。
或者存个模型。
'''
#[lab5]
#也要对他做stem #太久了。简直了 至少10-20分钟，太久了。 #该跑还是要跑。

neg_stem = []
for i in range(len(neg)):
  singles3 = []
  for plural in neg:
    singles3.append(stemmer.stem(plural))
  neg_stem.append(singles3)

print(neg_stem[:5])


pos_stem = []
for i in range(len(pos)):
  singles4 = []
  for plural in pos:
    singles4.append(stemmer.stem(plural))
  pos_stem.append(singles4)

print(pos_stem[:5])
